In [1]:
refined_template_path = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/Text/text_lib/Ideas/augmentation/data/musecoco_x_gpt_refined_template_en.json"   

verified_data_dir = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/Text/text_lib/Ideas/augmentation/data/verified"

In [2]:
gpt_refined_template_path = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/Text/text_lib/Ideas/augmentation/data/gpt_refined_template_en.json"

musecoco_refined_template_path = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/Text/text_lib/Ideas/augmentation/data/musecoco_refined_template_en.json"

In [3]:
midi_lib_path = "/Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/MIDI/midi_lib"

In [4]:
import sys

sys.path.append(midi_lib_path)

import const_lib.musecoco_const as mcc

In [5]:
import json

In [6]:
def load_json_file(file_path):
    with open(file_path, "r") as f:
        data = json.load(f)
    return data

In [7]:
# Concatenate all refined templates from MuseCoco and augmented data from GPT
musecoco_refined_template_data: list = load_json_file(musecoco_refined_template_path)

gpt_refined_template_data: list = load_json_file(gpt_refined_template_path)


print("MuseCoco refined template data loaded from: ", musecoco_refined_template_path)
print("MuseCoco refined template data: ", len(musecoco_refined_template_data))
print()
print("GPT refined template data loaded from: ", gpt_refined_template_path)
print("GPT refined template data: ", len(gpt_refined_template_data))
print()

concatenated_data = musecoco_refined_template_data + gpt_refined_template_data
print("Concatenated data: ", len(concatenated_data))

# Save the concatenated data to refined_template_path
with open(refined_template_path, "w") as f:
    json.dump(concatenated_data, f)

print("Concatenated data saved to: ", refined_template_path)

MuseCoco refined template data loaded from:  /Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/Text/text_lib/Ideas/augmentation/data/musecoco_refined_template_en.json
MuseCoco refined template data:  4856

GPT refined template data loaded from:  /Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/Text/text_lib/Ideas/augmentation/data/gpt_refined_template_en.json
GPT refined template data:  11300

Concatenated data:  16156
Concatenated data saved to:  /Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/Text/text_lib/Ideas/augmentation/data/musecoco_x_gpt_refined_template_en.json


In [8]:
gpt_refined_template = load_json_file(refined_template_path)

print(f"Total templates: {len(gpt_refined_template)}")

Total templates: 16156


In [9]:
def does_response_contain_required_substring_for_attribute(
    response: str, 
    attribute: str
) -> bool:
    """
        Check if the response contains the required substring for the attribute

        Args:
        - response (str): the response to check
        - attribute (str): the attribute to check

        Returns:
        - bool: True if the response contains the required substring for the attribute, False otherwise
    """
    result = False

    if attribute in mcc.required_substring_for_attributes:
        for substring in mcc.required_substring_for_attributes[attribute]:
            if substring.lower() in response.lower():
                result = True
                break

    if attribute in mcc.required_substring_for_positive_and_negative_attributes:
        for substring in mcc.required_substring_for_positive_and_negative_attributes[attribute]:
            if substring.lower() in response.lower():
                result = True and result
                break

    return result

In [10]:
def is_gpt_refined_template_acceptable(gpt_refined_template):
    attrs = gpt_refined_template["attributes"]  

    response = gpt_refined_template["response"]

    for attr in attrs:
        if not does_response_contain_required_substring_for_attribute(response, attr):
            print(f"Response does not contain required substring for attribute: {attr}, response: {response}")
            return False
        
    return True

In [11]:
def filter_acceptable_gpt_refined_templates(
    gpt_refined_templates: list
) -> dict[str, list]:
    acceptable_templates: list = []

    ignored_templates: list = []

    for gpt_refined_template in gpt_refined_templates:
        if is_gpt_refined_template_acceptable(gpt_refined_template):
            acceptable_templates.append(gpt_refined_template)
        else:
            ignored_templates.append(gpt_refined_template)

    return {
        "acceptable_templates": acceptable_templates,
        "ignored_templates": ignored_templates
    }

| **Ký hiệu** | **Mô tả**              |
|-------------|------------------------|
| I1s2        | Instrument             |
| I4          | Main Instrument        |
| R3          | Rhythm Intensity       |
| B1s1        | Bar                    |
| TS1s1       | Time Signature         |
| K1          | Key                    |
| T1s1        | Tempo                  |
| P4          | Pitch Range            |
| TM1         | Time                   |


In [12]:
# {
#     "I1_1": "[INSTRUMENTS] should be included in the music.", 
#     "I1_0": "[INSTRUMENTS] are not featured in this song.", 
#     "P4_1": "Its pitch range is within [RANGE] octaves.", 
#     "C1_0": "This is a song that has a bright feeling from the beginning to the end.", 
#     "C1_1": "This is a song that has a very gloomy feeling from the beginning to the end.", 
#     "C1_2": "The song begins bright and then turns dark.", 
#     "C1_3": "The song begins dark and then brightens up.", 
#     "R1_1": "This music is suitable for dancing.", 
#     "R1_0": "This music is not suitable for dancing.", 
#     "R3_1": "The beat of this song is extremely strong.", 
#     "R3_0": "This song has a very peaceful beat.", 
#     "R3_2": "This song has a moderate beat.", 
#     "S4_1": "The song belongs to the [GENRE] genre.", 
#     "S4_0": "The song does not fit into the conventions of [GENRE] style.", 
#     "S2_1": "The music is in the vein of [ARTIST].",
#     "S2_0": "The song does not conform to [ARTIST]'s typical sound.", 
#     "B1_1": "The song spans approximately [NUM_BARS] bars.", 
#     "TS1_1": "The music is in [TIME_SIGNATURE].", 
#     "TS1_o": "The time signature of this song is not commonly used.", 
#     "K1_1": "This music is composed in the [KEY] key.", 
#     "T1_0": "The tempo of this song is rapid.", 
#     "T1_1": "The tempo of this song is slow.", 
#     "T1_2": "The tempo of this song is moderate.", 
#     "EM1_1": "The music conveys [EMOTION].", 
#     "TM1_1": "This song has a duration of [TM1] seconds."
# }

In [13]:
acceptable_templates_file_path = f"{verified_data_dir}/musecoco_x_gpt_acceptable_templates_en.json"
ignored_templates_file_path = f"{verified_data_dir}/musecoco_x_gpt_ignored_templates_en.json"

filtered_gpt_refined_templates = filter_acceptable_gpt_refined_templates(gpt_refined_template)

Response does not contain required substring for attribute: I4_0, response: This music is characterized by its limited pitch range of [RANGE] octaves, which allows for a greater emphasis on the nuances of tone and phrasing. The use of the [KEY] key in this music provides a powerful and memorable sound that is complemented by the calming and soothing beat, as the song plays for [TM1] seconds. The music is brought to life through the use of [INSTRUMENTS], while the meter of the music is [TIME_SIGNATURE]. It's worth noting that the melody track does not incorporate the use of [INSTRUMENT]. Despite its slow pace, this music has a unique and distinct quality that is sure to captivate its listeners.
Response does not contain required substring for attribute: I4_0, response: The [RANGE]-octave pitch range in this [GENRE]-style music delivers a powerful and focused performance that is further enhanced by the rich and dynamic sonic palette of the [KEY] key. The rhythm of this [TM1]-second track

In [14]:
acceptable_templates = filtered_gpt_refined_templates["acceptable_templates"]
print(f"Accept {len(acceptable_templates)} templates")

ignored_templates = filtered_gpt_refined_templates["ignored_templates"]
print(f"Ignore {len(ignored_templates)} templates")

Accept 15934 templates
Ignore 222 templates


In [15]:
acceptable_templates_with_unique_responses: list = []

already_existing_responses: set = set()

for acceptable_template in acceptable_templates:
    response = acceptable_template["response"]

    if response not in already_existing_responses:
        already_existing_responses.add(response)

        acceptable_templates_with_unique_responses.append(acceptable_template)

acceptable_templates = acceptable_templates_with_unique_responses

print(f"Total templates: {len(gpt_refined_template)}")
print("Total acceptable templates with unique responses: ", len(acceptable_templates))

Total templates: 16156
Total acceptable templates with unique responses:  14932


In [16]:
with open(acceptable_templates_file_path, "w") as f:
    json.dump(acceptable_templates, f, indent=4)

print(f"All acceptable templates: {len(acceptable_templates)}")

print(f"File path: {acceptable_templates_file_path}")

All acceptable templates: 14932
File path: /Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/Text/text_lib/Ideas/augmentation/data/verified/musecoco_x_gpt_acceptable_templates_en.json


In [17]:

with open(ignored_templates_file_path, "w") as f:
    json.dump(ignored_templates, f, indent=4)

print(f"Ignored templates (except duplicated): {len(ignored_templates)}")

print(f"File path: {ignored_templates_file_path}")

Ignored templates (except duplicated): 222
File path: /Users/4rr311/Documents/VectorA/KHTN/Nam4/HKII/Thesis/Brainstorming/Text/text_lib/Ideas/augmentation/data/verified/musecoco_x_gpt_ignored_templates_en.json
